In [0]:
import torch
import os
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
import torchvision
import pandas as pd
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
file_id = '1cnbAqbH8zg55VQHAF__umgjCaOrg4q-0' 
downloaded = drive.CreateFile({'id': file_id})
download = downloaded.GetContentFile('Base.7z')

In [0]:
!mkdir rootF
!7z x Base.7z -orootF


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 2518807395 bytes (2403 MiB)

Extracting archive: Base.7z
--
Path = Base.7z
Type = 7z
Physical Size = 2518807395
Headers Size = 11415
Method = LZMA2:192m
Solid = +
Blocks = 3

  0%      0% 1 - Base/20051019_38557_0100_PP.tif                                          0% 3 - Base/20051020_43832_0100_PP.tif                                          0% 4 - Base/20051020_43882_0100_PP.tif                                          0% 5 - Base/20051020_43906_0100_PP.tif

In [0]:
import gc
gc.collect()
csvfile = "/content/drive/My Drive/Training/Annotationbase123.csv"
rootdir = "/content/rootF/Base/"
trans = transforms.Compose([transforms.Resize((512,512)),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


In [0]:
class DRDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.fundus_samples = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.fundus_samples)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        sample = Image.open(rootdir+self.fundus_samples.values[idx][0])
        
        if self.transform:
            sample = self.transform(sample)

        return sample,self.fundus_samples.values[idx][2]

In [0]:
def displayImage(img):                   ###only works for single image
    _img = img[0].numpy()
    plt.imshow(np.transpose(_img,(1,2,0)), interpolation='nearest')
    plt.show()

In [0]:
futile = pd.read_csv(csvfile)
futile.head()


,Image name,Ophthalmologic department,Retinopathy grade,Risk of macular edema
0,20051019_38557_0100_PP.tif,Service Ophtalmologie Lariboisiere,3,1
1,20051020_43808_0100_PP.tif,Service Ophtalmologie Lariboisiere,0,0
2,20051020_43832_0100_PP.tif,Service Ophtalmologie Lariboisiere,1,0
3,20051020_43882_0100_PP.tif,Service Ophtalmologie Lariboisiere,2,0
4,20051020_43906_0100_PP.tif,Service Ophtalmologie Lariboisiere,3,2


In [0]:
im = Image.open(rootdir+futile.values[0][0])
# im.show()
# displayImage(im)

In [0]:
from sklearn.model_selection import train_test_split
dataset = DRDataset(csvfile,rootdir,trans)
trn, tst=train_test_split(dataset,train_size=0.8)
trainloader = torch.utils.data.DataLoader(trn, batch_size=16,
                                          shuffle=True, num_workers=8,pin_memory=True)
testloader = torch.utils.data.DataLoader(tst, batch_size=1,
                                          shuffle=True, num_workers=0,pin_memory=True)

dataiter = iter(trainloader)


In [0]:
_, vld=train_test_split(trn,train_size=0.8)
validationloader = torch.utils.data.DataLoader(vld, batch_size=1,
                                          shuffle=True, num_workers=0,pin_memory=True)

In [0]:
import torch.nn as nn
model = torch.hub.load('pytorch/vision:v0.6.0', 'googlenet', pretrained=False)
import torch.optim as optim


Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.6.0


In [0]:
gc.collect()
def get_device():
  if torch.cuda.is_available():
    device = 'cuda:0'
  else:
    device = 'cpu'
  return device
device = get_device()
print(device)

cuda:0


In [0]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
model.eval()
model.to(device)


GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [0]:
print(len(trainloader))

75


In [0]:
for epoch in range(15):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)
#         print("retreuved from iter")
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
#         print("passed from net")
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
#         print("stepped suceess")
        # print statistics
        running_loss += loss.item()
        if i % 10 == 9:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 10))
            running_loss = 0.0

print('Finished Training')

[1,    10] loss: 6.903
[1,    20] loss: 6.894
[1,    30] loss: 6.861
[1,    40] loss: 6.620
[1,    50] loss: 4.160
[1,    60] loss: 1.590
[2,    10] loss: 1.327
[2,    20] loss: 1.263
[2,    30] loss: 1.370
[2,    40] loss: 1.354
[2,    50] loss: 1.312
[2,    60] loss: 1.262
[3,    10] loss: 1.337
[3,    20] loss: 1.271
[3,    30] loss: 1.279
[3,    40] loss: 1.313
[3,    50] loss: 1.285
[3,    60] loss: 1.275
[4,    10] loss: 1.315
[4,    20] loss: 1.252
[4,    30] loss: 1.309
[4,    40] loss: 1.262
[4,    50] loss: 1.260
[4,    60] loss: 1.342
[5,    10] loss: 1.311
[5,    20] loss: 1.271
[5,    30] loss: 1.265
[5,    40] loss: 1.331
[5,    50] loss: 1.256
[5,    60] loss: 1.292
[6,    10] loss: 1.260
[6,    20] loss: 1.278
[6,    30] loss: 1.357
[6,    40] loss: 1.264
[6,    50] loss: 1.291
[6,    60] loss: 1.305
[7,    10] loss: 1.278
[7,    20] loss: 1.352
[7,    30] loss: 1.303
[7,    40] loss: 1.299
[7,    50] loss: 1.235
[7,    60] loss: 1.252
[8,    10] loss: 1.237
[8,    20] 

In [0]:
PATH = '/content/drive/My Drive/Training/dr3.pth'
torch.save(model.state_dict(), PATH)

In [0]:
model = torch.hub.load('pytorch/vision:v0.6.0', 'googlenet', pretrained=False)
PATH = '/content/drive/My Drive/Training/dr.pth'
model.load_state_dict(torch.load(PATH))


Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.6.0


<All keys matched successfully>

In [0]:
model.eval()
model.to("cpu")

GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [0]:
dataiter = iter(testloader)
images = 0
accuracy = 0
for epoch in range(1):  # loop over the dataset multiple times

    for i, data in enumerate(testloader, 0):
        images+=1
        inputs, labels = data[0].to("cpu"), data[1].to("cpu")
        # print(np.shape(inputs.data.detach().numpy()))
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        # print(predicted,labels)
        accuracy += (predicted == labels).sum().item()

        # if labels == inputs:
        #   accuracy+=1
        # print statistics
        # if i % 20 == 19:    # print every 2000 mini-batches
        #     print('[%d] accuracy: %.3f' %
        #           (i + 1, accuracy / images))
        #     running_loss = 0.0
print(accuracy/images)
print('Finished Testing')

0.48333333333333334
Finished Evaluation


In [0]:
dataiter = iter(validationloader)
images = 0
accuracy = 0
for epoch in range(1):  # loop over the dataset multiple times

    for i, data in enumerate(validationloader, 0):
        images+=1
        inputs, labels = data[0].to("cpu"), data[1].to("cpu")
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        accuracy += (predicted == labels).sum().item()

        # if labels == inputs:
        #   accuracy+=1
        # print statistics
        # if i % 20 == 19:    # print every 2000 mini-batches
        #     print('[%d] accuracy: %.3f' %
        #           (i + 1, accuracy / images))
        #     running_loss = 0.0
print(accuracy/images)
print('Finished Evaluation')

0.453125
Finished Evaluation


In [0]:
# img = dataiter.next()[0]
# displayImage(img)


In [0]:
class testIMDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, listNm, root_dir, transform=None):
        self.fundus_samples = listNm
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.fundus_samples)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        sample = Image.open(rootdir+self.fundus_samples[idx])
        if self.transform:
            sample = self.transform(sample)
        return sample

def detectStage(model,imgnamelist,rootdir,trans):
  model.eval()
  # model.to(device)
  dse = testIMDataset(imgnamelist,rootdir,trans)
  dseloader = torch.utils.data.DataLoader(dse, batch_size=1,
                                            shuffle=False, num_workers=0)
  dataiter = iter(dseloader)
  image= dataiter.next()
  output = model(image)
  _, predicted = torch.max(output.data, 1)
  return(predicted.detach().numpy()[0])

In [0]:
detectStage(model,[futile.values[128][0]],rootdir,trans)

NameError: ignored